In [ ]:
# Activate logging
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [ ]:
from getpass import getpass

openai_api_key = getpass("Enter OpenAI API key:")

In [ ]:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore(
    similarity="cosine",
    index="document",
    embedding_field="embedding",
    embedding_dim=1536
)

In [ ]:
from datasets import load_dataset

# load seven wonders dataset from datasets and write documents into elasticsearch document store

dataset = load_dataset("bilgeyucel/seven-wonders", split="train")
document_store.write_documents(dataset)

In [ ]:
from haystack.nodes import EmbeddingRetriever, OpenAIAnswerGenerator

# define retriever model , update dense vector embeddings on document store index and define generator model
retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="text-embedding-ada-002",
    api_key=openai_api_key,
    max_seq_len=1024,
    top_k=5,
)
document_store.update_embeddings(retriever=retriever)
generator = OpenAIAnswerGenerator(api_key=openai_api_key,
                                  model="text-davinci-003",
                                  temperature=0.2)

In [ ]:
from haystack import Pipeline

pipe = Pipeline()
pipe.add_node(component=retriever, name="Retriever", inputs=["Query"])
pipe.add_node(component=generator, name="Generator", inputs=["Retriever"])

In [ ]:
from haystack_memory.prompt_templates import memory_template
from haystack.agents import Agent, Tool
from haystack.nodes import PromptNode

# create agent prompt node and define our Agent "memory_agent"
prompt_node = PromptNode(model_name_or_path="text-davinci-003", api_key=openai_api_key, max_length=512,
                         stop_words=["Observation:"])
memory_agent = Agent(prompt_node=prompt_node, prompt_template=memory_template)
# Define the first tool: A document store QA tool based on the pipeline
search_tool = Tool(name="DocumentStore_QA",
                   pipeline_or_node=pipe,
                   description="Access this tool to find out missing information needed to answer questions",
                   output_variable="answers")

In [ ]:
from haystack_memory.memory import RedisMemoryRecallNode

redis_memory_node = RedisMemoryRecallNode(memory_id="agent_memory",
                                          host="localhost",
                                          port=6379,
                                          db=0)
memory_tool = Tool(name="Memory",
                   pipeline_or_node=redis_memory_node,
                   description="Your memory. Always access this tool first to remember what you have learned.")

In [ ]:
# Add the memory and the search tool to the agent
memory_agent.add_tool(search_tool)
memory_agent.add_tool(memory_tool)

In [ ]:
from haystack_memory.utils import RedisUtils

# Chat with the agent
redis_utils = RedisUtils(agent=memory_agent,
                         memory_id="agent_memory",
                         host= "localhost",
                         port=6379,
                         db=0)
redis_utils.chat("What does the Rhodes Statue look like?")

In [ ]:
redis_utils.chat("What is the color of the spikes?")